In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath

from lib.sych.data_fc_db_raw import DataFCDatabase

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

In [ ]:
dataLst = dataDB.get_neuro_data({'mousename' : 'mvg_4'}, trialType='iGO')

In [ ]:
dataLst[0].shape

## 1. AR(1) Model

In [ ]:
from mesostat.metric.impl.basis_projector import BasisProjector
from scipy.stats import f
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
def ftest(y, yhat1, yhat2, k1, k2):
    n = np.prod(y.shape)
    RSS1 = np.sum((y - yhat1)**2)
    RSS2 = np.sum((y - yhat2)**2)
    F = (1 - RSS2/RSS1) * (n - k2) / (k2 - k1)
    pval = 1 - f.cdf(F, k2-k1, n-k2)
    return F, pval
    
def model_AR(dataRS, n):
    nTrial, nTime = dataRS.shape
    model = LinearRegression(fit_intercept=False)
    x = np.array([dataRS[:, i:i-n] for i in range(n)]).transpose((1,2,0)).reshape((nTrial * (nTime-n)), n)
    
    y = dataRS[:, n:].flatten()
    model.fit(x, y)
    yhat = model.predict(x)
    
#     print({'nHist' : n, 'coef' : model.coef_, 'R2' : r2_score(y, yhat)})
    return yhat.reshape((nTrial, nTime-n))

nHistMax = 10
y = dataLst[0][:, :, 0]
yhat0 = np.full(y.shape, np.mean(y))
yhat = [yhat0] + [model_AR(y, i) for i in range(1, nHistMax+1)]
fLst = [[i, i+1, *ftest(y[:, i+1:], yhat[i][:, 1:], yhat[i+1], i, i+1)] for i in range(nHistMax)]

pd.DataFrame(fLst, columns=['df1', 'df2', 'F', 'pval'])

## 2. Legendre Poly

In [ ]:
basis = BasisProjector(200, order=5).polysample

plt.figure()
plt.plot(basis.T)
plt.show()

In [ ]:
legendre(0)

In [ ]:
def model_legendre(dataRS, n):
    nTrial, nTime = dataRS.shape
    
    basis = BasisProjector(nTime, order=n)
    basis.project(np.mean(dataRS, axis=0))
    yhat = basis.predict()
    yhat = np.outer(np.ones(nTrial), yhat)
    
#     print({'nHist' : n, 'coef' : basis.coeff, 'R2' : r2_score(dataRS.flatten(), yhat.flatten())})
    return yhat

y = dataLst[0][:, :, 0]
model_legendre(y, 0)

In [ ]:
nBasisMax = 20
yhat = [model_legendre(y, i) for i in range(nBasisMax+1)]
fLst = [[i, i+1, r2_score(y.flatten(), yhat[i+1].flatten()),
         *ftest(y, yhat[i], yhat[i+1], i+1, i+2)] for i in range(nBasisMax)]

pd.DataFrame(fLst, columns=['df1', 'df2', 'R2', 'F', 'pval'])